In [ ]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [2]:
import pandas as pd 
data = pd.read_csv('text_emotion.csv', nrows = 10000)

In [3]:
data = data[['sentiment', 'content']]
labels = data.sentiment 
texts = data.content
data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
import re
texts = texts.apply(lambda x: x.lower())
texts = texts.apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


from nltk.corpus import stopwords
stop=stopwords.words('english')
texts=texts.apply(lambda sen:" ".join(x for x in sen.split() if x not in stop))
texts.head()

0    tiffanylue know listenin bad habit earlier sta...
1               layin n bed headache ughhhhwaitin call
2                        funeral ceremonygloomy friday
3                              wants hang friends soon
4    dannycastillo want trade someone houston ticke...
Name: content, dtype: object

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras import metrics
import numpy as np


In [6]:
MAX_NB_WORDS = 2000

Y = data['sentiment']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
Y=le.transform(Y) 
labels = to_categorical(np.asarray(Y))
tok = Tokenizer(num_words=MAX_NB_WORDS)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(mat_texts,labels, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6700, 2000) (6700, 13)
(3300, 2000) (3300, 13)


In [8]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(MAX_NB_WORDS,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(13, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                128064    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 13)                845       
Total params: 133,069
Trainable params: 133,069
Non-trainable params: 0
_______________

In [9]:
from keras import models
from keras import layers
def build_model():
    
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(MAX_NB_WORDS,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(13, activation='softmax'))
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics = ['accuracy'])
    return(model)

In [10]:
import numpy as np
num_val_samples = 6700

k = 4
num_val_samples = len(X_train) // k
num_epochs = 5
all_scores = []
for i in range(k):
    print('processing fold #', i)
    
    val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = Y_train[i * num_val_samples: (i + 1) * num_val_samples]

   
    partial_train_data = np.concatenate(
        [X_train[:i * num_val_samples],
         X_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [Y_train[:i * num_val_samples],
         Y_train[(i + 1) * num_val_samples:]],
        axis=0)

    
    model = build_model()
    
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
Instructions for updating:
Use tf.cast instead.
processing fold # 1
processing fold # 2
processing fold # 3


In [11]:
print(np.mean(all_scores))

0.2698507463398264


In [12]:
model = build_model()

model.fit(X_train, Y_train,
          epochs=10, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(X_test, Y_test)

3300/3300 [==============================] - 0s 67us/step


In [13]:
test_mae_score

0.26242424242424245